In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [11]:
load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'

load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'

data_types = ['Wake','N1', 'N2', 'N3', 'REM']

for data_type in data_types:
    
    X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
    y_bpw = pd.read_hdf(load_path_bpw + data_type +  '_y.h5', key='df', mode='r') 
    groups_bpw = pd.read_hdf(load_path_bpw + data_type +  '_groups.h5', key='df', mode='r')
    X_bpw, y_bpw , groups_bpw = X_bpw.reset_index(drop = True) , y_bpw.reset_index(drop = True) , groups_bpw.reset_index(drop = True)

    # try:
    #     print(y_bpw.equals(y_max_freqs))
    #     print(groups_bpw.equals(groups_max_freqs))
    # except:
    #     pass
     
    X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
    y_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  '_y.h5', key='df', mode='r') 
    groups_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  '_groups.h5', key='df', mode='r')
    X_max_freqs, y_max_freqs, groups_max_freqs = X_max_freqs.reset_index(drop = True) , y_max_freqs.reset_index(drop = True) , groups_max_freqs.reset_index(drop = True)

    print(X_max_freqs.equals(X_bpw))
    print(y_bpw.equals(y_max_freqs))
    print(groups_bpw.equals(groups_max_freqs))


False
True
True
False
True
True
False
True
True
False
True
True
False
True
True


In [16]:
pd.concat([X_bpw , X_max_freqs], axis = 1)

Delta_Central  \
0   0      0.534534
1      0.387710
2      0.42522...   
1   0      0.424105
1      0.477326
2      0.48773...   
2   0      0.472426
1      0.511446
2      0.36640...   
3   0      0.525534
1      0.684082
2      0.54797...   
4   0      0.638859
1      0.475210
2      0.62820...   
..                                                ...   
64  0      0.751172
1      0.572066
2      0.55140...   
65  0      0.467329
1      0.408695
2      0.22749...   
66  0      0.530315
1      0.347132
2      0.35203...   
67  0      0.515000
1      0.556958
2      0.62824...   
68  0      0.519450
1      0.477623
2      0.69260...   

                                        Theta_Central  \
0   0      0.293270
1      0.424379
2      0.26447...   
1   0      0.243771
1      0.368189
2      0.31542...   
2   0      0.372754
1      0.302115
2      0.40864...   
3   0      0.256615
1      0.162816
2      0.29349...   
4   0      0.254707
1      0.395107
2      0.18505...   
..                                                ...   
64  0      0.103529
1      0.254530
2      0.19959...   
65  0      0.183017
1      0.093873
2      0.16797...   
66  0      0.109526
1      0.121884
2      0.09047...   
67  0      0.188657
1      0.165721
2      0.11952...   
68  0      0.355375
1      0.333536
2      0.20276...   

                                        Alpha_Central  \
0   0      0.086945
1      0.067723
2      0.11175...   
1   0      0.186910
1      0.060934
2      0.07706...   
2   0      0.099389
1      0.111395
2      0.18949...   
3   0      0.143287
1      0.087171
2      0.07770...   
4   0      0.065689
1      0.058731
2      0.07480...   
..                                                ...   
64  0      0.071460
1      0.105147
2      0.12524...   
65  0      0.088979
1      0.086744
2      0.16675...   
66  0      0.044845
1      0.138917
2      0.04452...   
67  0      0.075175
1      0.089367
2      0.11108...   
68  0      0.052849
1      0.126598
2      0.05069...   

                                        Sigma_Central  \
0   0      0.039545
1      0.053726
2      0.10141...   
1   0      0.057856
1      0.033569
2      0.02528...   
2   0      0.027505
1      0.031913
2      0.01899...   
3   0      0.030819
1      0.041268
2      0.03733...   
4   0      0.021363
1      0.027092
2      0.05270...   
..                                                ...   
64  0      0.035066
1      0.024258
2      0.06416...   
65  0      0.030358
1      0.036878
2      0.06972...   
66  0      0.051056
1      0.060858
2      0.04254...   
67  0      0.065378
1      0.100472
2      0.03983...   
68  0      0.023183
1      0.030577
2      0.02122...   

                                         Beta_Central  \
0   0      0.034052
1      0.051924
2      0.08084...   
1   0      0.075055
1      0.040070
2      0.04842...   
2   0      0.023498
1      0.038645
2      0.01395...   
3   0      0.038566
1      0.021959
2      0.03689...   
4   0      0.016941
1      0.037342
2      0.05253...   
..                                                ...   
64  0      0.033570
1      0.037580
2      0.05571...   
65  0      0.150135
1      0.284568
2      0.27455...   
66  0      0.161470
1      0.215923
2      0.32289...   
67  0      0.149861
1      0.080725
2      0.09404...   
68  0      0.035928
1      0.026285
2      0.02755...   

                                        Gamma_Central  \
0   0      0.011654
1      0.014538
2      0.01627...   
1   0      0.012304
1      0.019910
2      0.04607...   
2   0      0.004429
1      0.004486
2      0.00251...   
3   0      0.005179
1      0.002703
2      0.00659...   
4   0      0.002441
1      0.006518
2      0.00669...   
..                                                ...   
64  0      0.005204
1      0.006420
2      0.00386...   
65  0      0.080183
1      0.089243
2      0.09348...   
66  0      0.102788
1      0.115286
2      0.14752...   
67  0      0.005929
1      0.006757
2      0.